<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#FSA-Values" data-toc-modified-id="FSA-Values-1">FSA Values</a></span></li><li><span><a href="#Statistics-Canada-API-Preview" data-toc-modified-id="Statistics-Canada-API-Preview-2">Statistics Canada API Preview</a></span></li><li><span><a href="#Methodology" data-toc-modified-id="Methodology-3">Methodology</a></span></li><li><span><a href="#Visible-Minorities-Data" data-toc-modified-id="Visible-Minorities-Data-4">Visible Minorities Data</a></span><ul class="toc-item"><li><span><a href="#Data-On-One-FSA-Value" data-toc-modified-id="Data-On-One-FSA-Value-4.1">Data On One FSA Value</a></span></li><li><span><a href="#Data-On-All-FSA-Values" data-toc-modified-id="Data-On-All-FSA-Values-4.2">Data On All FSA Values</a></span></li><li><span><a href="#Cleaning-Visible-Minorities-Data" data-toc-modified-id="Cleaning-Visible-Minorities-Data-4.3">Cleaning Visible Minorities Data</a></span></li><li><span><a href="#Final-Visible-Minorities-Data" data-toc-modified-id="Final-Visible-Minorities-Data-4.4">Final Visible Minorities Data</a></span></li></ul></li><li><span><a href="#Population-Data" data-toc-modified-id="Population-Data-5">Population Data</a></span><ul class="toc-item"><li><span><a href="#Data-On-One-FSA-Value" data-toc-modified-id="Data-On-One-FSA-Value-5.1">Data On One FSA Value</a></span></li><li><span><a href="#Data-On-All-FSA-Values" data-toc-modified-id="Data-On-All-FSA-Values-5.2">Data On All FSA Values</a></span></li><li><span><a href="#Cleaning-Population-Data" data-toc-modified-id="Cleaning-Population-Data-5.3">Cleaning Population Data</a></span></li><li><span><a href="#Final-Population-Data" data-toc-modified-id="Final-Population-Data-5.4">Final Population Data</a></span></li></ul></li><li><span><a href="#Income-Data" data-toc-modified-id="Income-Data-6">Income Data</a></span><ul class="toc-item"><li><span><a href="#Data-On-One-FSA-Value" data-toc-modified-id="Data-On-One-FSA-Value-6.1">Data On One FSA Value</a></span></li><li><span><a href="#Data-On-All-FSA-Values" data-toc-modified-id="Data-On-All-FSA-Values-6.2">Data On All FSA Values</a></span></li><li><span><a href="#Final-Income-Data" data-toc-modified-id="Final-Income-Data-6.3">Final Income Data</a></span></li></ul></li><li><span><a href="#Commute-Data" data-toc-modified-id="Commute-Data-7">Commute Data</a></span><ul class="toc-item"><li><span><a href="#Data-On-One-FSA-Value" data-toc-modified-id="Data-On-One-FSA-Value-7.1">Data On One FSA Value</a></span></li><li><span><a href="#Data-On-All-FSA-Values" data-toc-modified-id="Data-On-All-FSA-Values-7.2">Data On All FSA Values</a></span></li><li><span><a href="#Final-Commute-Data" data-toc-modified-id="Final-Commute-Data-7.3">Final Commute Data</a></span></li></ul></li><li><span><a href="#Immigration-Data" data-toc-modified-id="Immigration-Data-8">Immigration Data</a></span><ul class="toc-item"><li><span><a href="#Data-On-One-FSA-Value" data-toc-modified-id="Data-On-One-FSA-Value-8.1">Data On One FSA Value</a></span></li><li><span><a href="#Data-On-All-FSA-Value" data-toc-modified-id="Data-On-All-FSA-Value-8.2">Data On All FSA Value</a></span></li><li><span><a href="#Final-Immigration-Data" data-toc-modified-id="Final-Immigration-Data-8.3">Final Immigration Data</a></span></li></ul></li><li><span><a href="#Housing-Data" data-toc-modified-id="Housing-Data-9">Housing Data</a></span><ul class="toc-item"><li><span><a href="#Data-On-One-FSA-Value" data-toc-modified-id="Data-On-One-FSA-Value-9.1">Data On One FSA Value</a></span></li><li><span><a href="#Data-On-All-FSA-Values" data-toc-modified-id="Data-On-All-FSA-Values-9.2">Data On All FSA Values</a></span></li><li><span><a href="#Final-Housing-Data" data-toc-modified-id="Final-Housing-Data-9.3">Final Housing Data</a></span></li></ul></li><li><span><a href="#Combining-All-Demographic-Data" data-toc-modified-id="Combining-All-Demographic-Data-10">Combining All Demographic Data</a></span></li><li><span><a href="#Export-Data" data-toc-modified-id="Export-Data-11">Export Data</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import requests 
import json
from pandas import json_normalize

# FSA Values

Firstly, we need a list of all the FSA's we will be adding to our dataset. 

In [2]:
r_model_data = pd.read_csv("cleaned_rest_data.csv")
list_of_FSA = list(r_model_data['FSA'].value_counts().index)
len(list_of_FSA)

98

The list of FSA values have 98 values, and should only have FSA's that begin with `M` or `L`.

# Statistics Canada API Preview

The data is from Statistics Canada. It comes in the form of an API.

The main part of the API comes from the [2016 Census Profile Web Data Service](https://www12.statcan.gc.ca/wds-sdw/cpr2016-eng.cfm). This API service interprets a single FSA value. 

Parameters:
- **type**: json or xml formats.
- **lang**: french or english (E/F)
- **dguid**: alphanumeric code.
    - In our dataset, the format of the `dguid` will be
    - "2016A0011xxx" where xxx represents the FSA
- **topic**: list of possible data points
    - (options from 0 to 14, 0 denoting all topics)
- **notes**: include footnotes (0 = do not include)
- **stat** where 0 = counts, 1 = rates

Resource URL:

https://www12.statcan.gc.ca/rest/census-recensement/CPR2016{.type}?lang={lang}&dguid={dguid}&topic={topic}&notes={notes}&stat={stat}

# Methodology

To gather data from various sources in the Statistics Canada API, I will gather datasets for the following themes:

- Visible Minority Data
- Population Data
- Education Data
- Ethnic Origin
- Immigration
- Income

Each dataset should have 103 rows, with each row representing an FSA. For each dataset, I will first gather data on an arbitrary FSA value, and then create a for loop to get data from all FSA values. Then, I will combine it into a dataset, clean it up (without removing any rows; only removing or renaming columns), and then output the data.

After, I will combine all the datasets together to create a full demographics dataset, and then proceed to clean the data.


# Visible Minorities Data

## Data On One FSA Value

First, lets call the API and have my home postal code 'M4Y' as the parameter. The API returns a full table/dataframe for a single postal code. Lets see what it looks like:

In [3]:
# URL for census data
url = "https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json"

# Parameters for visible minority population; One postal code only
parameters = {
"lang": "E",                     # Requested Data in English
"dguid": "2016A0011" + "V7C",    # Geographic Region
"topic": 14,                     # 14 = Visible Minority
"notes": 0,                      # Foot Notes
"stat": 0                        # Count (0) or Percentages (1)
}


In [4]:

# Get response
response = requests.get(url, params = parameters)

# Load data as text (disregard first two // symbols)
x = json.loads(response.text[2:])['DATA']

# Save text data as dataframe (for V7C)
df = pd.DataFrame.from_records(x)

# Add columns to the dataframe (for V7C)
df.columns = json.loads(response.text[2:])['COLUMNS']

df.head()

,PROV_TERR_ID,PROV_TERR_NAME_NOM,GEO_UID,GEO_ID,GEO_NAME_NOM,GEO_TYPE,TOPIC_THEME,TEXT_ID,HIER_ID,INDENT_ID,TEXT_NAME_NOM,NOTE_ID,NOTE,T_DATA_DONNEE,T_SYM,M_DATA_DONNEE,M_SYM,F_DATA_DONNEE,F_SYM
0,59,British Columbia,2016A0011V7C,V7C,V7C,None,Visible minority,25000,7.1.1,0,Total - Visible minority for the population in...,95.0,None,39220.0,None,18680.0,None,20540.0,None
1,59,British Columbia,2016A0011V7C,V7C,V7C,None,Visible minority,25001,7.1.1.1,1,Total visible minority population,96.0,None,30370.0,None,14285.0,None,16085.0,None
2,59,British Columbia,2016A0011V7C,V7C,V7C,None,Visible minority,25002,7.1.1.1.1,2,South Asian,97.0,None,1820.0,None,920.0,None,900.0,None
3,59,British Columbia,2016A0011V7C,V7C,V7C,None,Visible minority,25003,7.1.1.1.2,2,Chinese,NaN,None,22935.0,None,10735.0,None,12200.0,None
4,59,British Columbia,2016A0011V7C,V7C,V7C,None,Visible minority,25004,7.1.1.1.3,2,Black,NaN,None,265.0,None,140.0,None,125.0,None


This is not quite what we wanted. We want a single row representing a whole FSA, with the information we want in a single table. Lets try converting this to what we want:

In [5]:
# Rows that we want to turn into column names
minority_column = df['TEXT_NAME_NOM'].values

# Actual data values we want to fit under the column names.
minority_data = df['T_DATA_DONNEE'] 

# Turn them into a row
minority_df = pd.DataFrame(minority_data).T

# Add column names
minority_df.columns = minority_column

minority_df

,Total - Visible minority for the population in private households - 25% sample data,Total visible minority population,South Asian,Chinese,Black,Filipino,Latin American,Arab,Southeast Asian,West Asian,Korean,Japanese,"Visible minority, n.i.e.",Multiple visible minorities,Not a visible minority
T_DATA_DONNEE,39220.0,30370.0,1820.0,22935.0,265.0,2275.0,345.0,395.0,320.0,260.0,250.0,630.0,95.0,790.0,8850.0


## Data On All FSA Values

Now, I will create a loop to do this for all the FSA's I want.

In [6]:
# Empty dataframe with the column names we want:
visible_minority_data = pd.DataFrame(columns = minority_column)

# URL for Statistics Canada API
url = "https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json"

# For every FSA value:
for fsa_code in list_of_FSA:
    # Update parameters for API request
    parameters = {
    "lang": "E",                     # Requested Data in English
    "dguid": "2016A0011" + fsa_code, # Geographic Region
    "topic": 14,                     # Visible Minority
    "notes": 0,                      # Foot Notes
    "stat": 0                        # Count (0) or Percentages (1)
    }
    
    # API Request
    response = requests.get(url, params = parameters)
    
    # Create temporary string to store dataframe from single FSA
    temp_string = json.loads(response.text[2:])['DATA']
    
    # Convert temporary string into a temporary dataframe
    temp_df = pd.DataFrame.from_records(temp_string)
    
    # Assigning columns in temporary dataframe
    temp_df.columns = json.loads(response.text[2:])['COLUMNS']
    
    # Creating a single row with data of a single FSA
    single_minority_row = temp_df['T_DATA_DONNEE']
    single_minority_row = pd.DataFrame(single_minority_row).T
    single_minority_row.columns = minority_column
    
    # Populating the dataframe with the single row
    visible_minority_data = visible_minority_data.append(single_minority_row, ignore_index = True)

# Add FSA value to to final table
visible_minority_data.insert(loc=0, column = "FSA", value = list_of_FSA)

print(f"Shape of Visible Minority Data: {visible_minority_data.shape}")

Shape of Visible Minority Data: (98, 16)


## Cleaning Visible Minorities Data

In [7]:
# Dropping column
visible_minority_data = visible_minority_data.drop(columns = ["Total - Visible minority for the population in private households - 25% sample data", \
                                                             '    Multiple visible minorities', '    Visible minority, n.i.e.'])

In [8]:
# Change column names
visible_minority_data.columns = ['FSA', 'Visible_Minority_Population_Pct', 'South_Asian_Pct', \
                                'Chinese_Pct', 'Black_Pct', 'Filipino_Pct', 'Latin American_Pct', \
                                'Arab_Pct', 'South_East_Asian_Pct', 'West_Asian_Pct', 'Korean_Pct', \
                                'Japanese_Pct', 'Not_Visible_Minority_Population_Pct']

## Final Visible Minorities Data

In [9]:
visible_minority_data.head()

,FSA,Visible_Minority_Population_Pct,South_Asian_Pct,Chinese_Pct,Black_Pct,Filipino_Pct,Latin American_Pct,Arab_Pct,South_East_Asian_Pct,West_Asian_Pct,Korean_Pct,Japanese_Pct,Not_Visible_Minority_Population_Pct
0,M5V,20745.0,4395.0,6290.0,2560.0,1020.0,1355.0,1000.0,605.0,1030.0,795.0,325.0,28040.0
1,M5T,10455.0,765.0,6045.0,880.0,450.0,330.0,290.0,615.0,190.0,280.0,215.0,6895.0
2,M6J,9055.0,900.0,4225.0,1100.0,440.0,615.0,170.0,335.0,150.0,320.0,185.0,22570.0
3,M6G,7605.0,740.0,2415.0,1120.0,570.0,720.0,155.0,355.0,155.0,475.0,195.0,23485.0
4,M5S,6450.0,1045.0,2980.0,355.0,210.0,225.0,265.0,145.0,205.0,610.0,170.0,7840.0


# Population Data

## Data On One FSA Value

In [10]:
# Parameters for population; One postal code only
parameters = {
"lang": "E",                     # Requested Data in English
"dguid": "2016A0011" + "V7C",    # Geographic Region
"topic": 13,                     # 13 = Population
"notes": 0,                      # Foot Notes
"stat": 0                        # Count (0) or Percentages (1)
}

# Get response
response = requests.get(url, params = parameters)

# Load data as text (disregard first two // symbols)
x = json.loads(response.text[2:])['DATA']

# Save text data as dataframe (for V7C)
df = pd.DataFrame.from_records(x)

# Add columns to the dataframe (for V7C)
df.columns = json.loads(response.text[2:])['COLUMNS']

# Rows that we want to turn into column names
single_data_columns = df['TEXT_NAME_NOM'].values

# Actual data values we want to fit under the column names.
single_data = df['T_DATA_DONNEE'] 

# Turn them into a row
single_data = pd.DataFrame(single_data).T

# Add column names
single_data.columns = single_data_columns

single_data

,"Population, 2016","Population, 2011","Population percentage change, 2011 to 2016",Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Total - Age groups and average age of the population - 100% data,0 to 14 years,0 to 4 years,...,90 to 94 years,95 to 99 years,100 years and over,Total - Distribution (%) of the population by broad age groups - 100% data,0 to 14 years,15 to 64 years,65 years and over,85 years and over,Average age of the population,Median age of the population
T_DATA_DONNEE,39252.0,NaN,NaN,14742.0,13958.0,NaN,NaN,39250.0,5630.0,1715.0,...,230.0,60.0,15.0,100.0,14.3,68.8,16.8,2.0,42.1,44.2


## Data On All FSA Values

In [11]:
# Empty dataframe with the column names we want:
population_data = pd.DataFrame(columns = single_data_columns)

# URL for Statistics Canada API
url = "https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json"

# For every FSA value:
for fsa_code in list_of_FSA:

    # Update parameters for API request
    parameters = {
    "lang": "E",                     # Requested Data in English
    "dguid": "2016A0011" + fsa_code, # Geographic Region
    "topic": 13,                     # Population = 13
    "notes": 0,                      # Foot Notes
    "stat": 0                        # Count (0) or Percentages (1)
    }
    
    # API Request
    response = requests.get(url, params = parameters)
    
    # Create temporary string to store dataframe from single FSA
    temp_string = json.loads(response.text[2:])['DATA']
    
    # Convert temporary string into a temporary dataframe
    temp_df = pd.DataFrame.from_records(temp_string)
    
    # Assigning columns in temporary dataframe
    temp_df.columns = json.loads(response.text[2:])['COLUMNS']
    
    # Creating a single row with data of a single FSA
    single_population_row = temp_df['T_DATA_DONNEE']
    single_population_row = pd.DataFrame(single_population_row).T
    single_population_row.columns = single_data_columns
    
    # Populating the dataframe with the single row
    population_data = population_data.append(single_population_row, ignore_index = True)
    

# Add FSA value to to final table
population_data.insert(loc=0, column = "FSA", value = list_of_FSA)

print(f"Shape of Population Data: {population_data.shape}")


Shape of Population Data: (98, 41)


## Cleaning Population Data

In [12]:
population_data.head(10)

,FSA,"Population, 2016","Population, 2011","Population percentage change, 2011 to 2016",Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Total - Age groups and average age of the population - 100% data,0 to 14 years,...,90 to 94 years,95 to 99 years,100 years and over,Total - Distribution (%) of the population by broad age groups - 100% data,0 to 14 years,15 to 64 years,65 years and over,85 years and over,Average age of the population,Median age of the population
0,M5V,49195.0,NaN,NaN,35058.0,30726.0,NaN,NaN,49195.0,2495.0,...,30.0,0.0,0.0,100.0,5.1,90.9,4.0,0.2,34.2,31.4
1,M5T,17748.0,NaN,NaN,9480.0,8537.0,NaN,NaN,17750.0,1315.0,...,150.0,25.0,10.0,100.0,7.4,77.2,15.4,2.7,40.3,34.3
2,M6J,32684.0,NaN,NaN,16333.0,15063.0,NaN,NaN,32685.0,3235.0,...,160.0,30.0,0.0,100.0,9.9,78.2,11.9,1.6,38.9,35.1
3,M6G,32086.0,NaN,NaN,14312.0,13424.0,NaN,NaN,32085.0,3800.0,...,390.0,135.0,25.0,100.0,11.8,70.6,17.6,4.1,41.8,38.2
4,M5S,15372.0,NaN,NaN,9380.0,7878.0,NaN,NaN,15370.0,865.0,...,130.0,50.0,10.0,100.0,5.6,78.9,15.5,2.9,39.0,31.6
5,M4Y,30472.0,NaN,NaN,21975.0,19665.0,NaN,NaN,30470.0,1335.0,...,110.0,25.0,0.0,100.0,4.4,85.1,10.5,1.3,39.1,34.0
6,M4M,24689.0,NaN,NaN,11393.0,10787.0,NaN,NaN,24690.0,3435.0,...,95.0,55.0,5.0,100.0,13.9,74.0,12.1,1.8,39.7,38.8
7,M6H,44950.0,NaN,NaN,20573.0,19209.0,NaN,NaN,44950.0,5315.0,...,195.0,45.0,5.0,100.0,11.8,75.4,12.8,1.6,39.5,36.4
8,M5A,41078.0,NaN,NaN,24186.0,22333.0,NaN,NaN,41080.0,4020.0,...,85.0,25.0,5.0,100.0,9.8,80.1,10.1,0.8,39.3,37.0
9,M5B,12785.0,NaN,NaN,8249.0,7058.0,NaN,NaN,12785.0,725.0,...,25.0,5.0,5.0,100.0,5.7,86.7,7.7,0.6,36.6,32.0


In [13]:
population_data.columns

Index(['FSA', 'Population, 2016', 'Population, 2011',
       'Population percentage change, 2011 to 2016', 'Total private dwellings',
       'Private dwellings occupied by usual residents',
       'Population density per square kilometre',
       'Land area in square kilometres',
       'Total - Age groups and average age of the population - 100% data',
       '  0 to 14 years', '    0 to 4 years', '    5 to 9 years',
       '    10 to 14 years', '  15 to 64 years', '    15 to 19 years',
       '    20 to 24 years', '    25 to 29 years', '    30 to 34 years',
       '    35 to 39 years', '    40 to 44 years', '    45 to 49 years',
       '    50 to 54 years', '    55 to 59 years', '    60 to 64 years',
       '  65 years and over', '    65 to 69 years', '    70 to 74 years',
       '    75 to 79 years', '    80 to 84 years', '    85 years and over',
       '      85 to 89 years', '      90 to 94 years', '      95 to 99 years',
       '      100 years and over',
       'Total - Distribu

In [14]:
# Removing columns
columns_to_drop = ['Population, 2011', 'Population percentage change, 2011 to 2016',
                   'Population density per square kilometre', 'Land area in square kilometres',
                  'Total - Age groups and average age of the population - 100% data', 
                  'Total - Distribution (%) of the population by broad age groups - 100% data',
                  '  0 to 14 years', '  15 to 64 years', '  65 years and over', '    85 years and over',
                  'Private dwellings occupied by usual residents', 'Total private dwellings']
population_data = population_data.drop(columns = columns_to_drop)

In [15]:

# List of columns except for 'FSA'
population_data_columns = list(population_data.columns)
population_data_columns.pop(0)

for column in population_data_columns:
    population_data[column] = pd.to_numeric(population_data[column])

In [16]:
population_data.columns

Index(['FSA', 'Population, 2016', '    0 to 4 years', '    5 to 9 years',
       '    10 to 14 years', '    15 to 19 years', '    20 to 24 years',
       '    25 to 29 years', '    30 to 34 years', '    35 to 39 years',
       '    40 to 44 years', '    45 to 49 years', '    50 to 54 years',
       '    55 to 59 years', '    60 to 64 years', '    65 to 69 years',
       '    70 to 74 years', '    75 to 79 years', '    80 to 84 years',
       '      85 to 89 years', '      90 to 94 years', '      95 to 99 years',
       '      100 years and over', 'Average age of the population',
       'Median age of the population'],
      dtype='object')

In [17]:
# Reclassifying population age groups

# 0-19
population_data['Age:0_to_19'] = population_data['    0 to 4 years'] + population_data['    5 to 9 years'] + \
                                population_data['    10 to 14 years'] + population_data['    15 to 19 years']

population_data = population_data.drop(columns = ['    0 to 4 years', '    5 to 9 years', \
                                                  '    10 to 14 years', '    15 to 19 years'])

# 20-39
population_data['Age:20_to_39'] = population_data['    20 to 24 years'] + population_data['    25 to 29 years'] + \
                                population_data['    30 to 34 years'] + population_data['    35 to 39 years']

population_data = population_data.drop(columns = ['    20 to 24 years', '    25 to 29 years', \
                                                  '    30 to 34 years', '    35 to 39 years'])

# 40-59
population_data['Age:40_to_59'] = population_data['    40 to 44 years'] + population_data['    45 to 49 years'] + \
                                population_data['    50 to 54 years'] + population_data['    55 to 59 years']

population_data = population_data.drop(columns = ['    40 to 44 years', '    45 to 49 years', \
                                                  '    50 to 54 years', '    55 to 59 years'])

# 60+
population_data['Age:60plus'] = population_data['    60 to 64 years'] + population_data['    65 to 69 years'] + \
                                population_data['    70 to 74 years'] + population_data['    75 to 79 years'] + \
                                population_data['    80 to 84 years'] + \
                                population_data['      85 to 89 years'] + population_data['      90 to 94 years'] + \
                                population_data['      95 to 99 years'] + population_data['      100 years and over']

population_data = population_data.drop(columns = ['    60 to 64 years', '    65 to 69 years', \
                                                  '    70 to 74 years', '    75 to 79 years', \
                                                  '    80 to 84 years',\
                                                  '      85 to 89 years', '      90 to 94 years', \
                                                  '      95 to 99 years', '      100 years and over', 'Median age of the population' ])
population_data.columns = ['FSA', 'Population', 'Population_Average_Age', 'Age:0-19', 'Age:20-39', 'Age:40-59', 'Age:60plus']

## Final Population Data

In [18]:
population_data.head()

,FSA,Population,Population_Average_Age,Age:0-19,Age:20-39,Age:40-59,Age:60plus
0,M5V,49195.0,34.2,3215.0,34130.0,8815.0,3045.0
1,M5T,17748.0,40.3,2035.0,8230.0,3945.0,3535.0
2,M6J,32684.0,38.9,4060.0,15465.0,7935.0,5210.0
3,M6G,32086.0,41.8,4935.0,11855.0,8095.0,7190.0
4,M5S,15372.0,39.0,1740.0,7870.0,2735.0,3015.0


# Income Data

## Data On One FSA Value

In [19]:
# Parameters for Income; One postal code only
parameters = {
"lang": "E",                     # Requested Data in English
"dguid": "2016A0011" + "V7C",    # Geographic Region
"topic": 7,                     # 7 = Income
"notes": 0,                      # Foot Notes
"stat": 0                        # Count (0) or Percentages (1)
}

# Get response
response = requests.get(url, params = parameters)

# Load data as text (disregard first two // symbols)
x = json.loads(response.text[2:])['DATA']

# Save text data as dataframe (for V7C)
df = pd.DataFrame.from_records(x)

# Add columns to the dataframe (for V7C)
df.columns = json.loads(response.text[2:])['COLUMNS']

# Rows that we want to turn into column names
single_data_columns = df['TEXT_NAME_NOM'].values

# Actual data values we want to fit under the column names.
single_data = df['T_DATA_DONNEE'] 

# Turn them into a row
single_data = pd.DataFrame(single_data).T

# Add column names
single_data.columns = single_data_columns

single_data

,Total - Income statistics in 2015 for the population aged 15 years and over in private households - 100% data,Number of total income recipients aged 15 years and over in private households - 100% data,Median total income in 2015 among recipients ($),Number of after-tax income recipients aged 15 years and over in private households - 100% data,Median after-tax income in 2015 among recipients ($),Number of market income recipients aged 15 years and over in private households - 100% data,Median market income in 2015 among recipients ($),Number of government transfers recipients aged 15 years and over in private households - 100% data,Median government transfers in 2015 among recipients ($),Number of employment income recipients aged 15 years and over in private households - 100% data,...,"In low income based on the Low-income cut-offs, after tax (LICO-AT)",0 to 17 years,0 to 5 years,18 to 64 years,65 years and over,"Prevalence of low income based on the Low-income cut-offs, after tax (LICO-AT) (%)",0 to 17 years (%),0 to 5 years (%),18 to 64 years (%),65 years and over (%)
T_DATA_DONNEE,33430.0,31745.0,24602.0,31795.0,22994.0,28140.0,24948.0,23090.0,1503.0,21115.0,...,8050.0,1875.0,540.0,5380.0,795.0,20.6,26.4,26.3,21.1,12.3


## Data On All FSA Values

In [20]:
# Empty dataframe with the column names we want:
income_data = pd.DataFrame(columns = single_data_columns)

# URL for Statistics Canada API
url = "https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json"

# For every FSA value:
for fsa_code in list_of_FSA:

    # Update parameters for API request
    parameters = {
    "lang": "E",                     # Requested Data in English
    "dguid": "2016A0011" + fsa_code, # Geographic Region
    "topic": 7,                     # Income
    "notes": 0,                      # Foot Notes
    "stat": 0                        # Count (0) or Percentages (1)
    }
    
    # API Request
    response = requests.get(url, params = parameters)
    
    # Create temporary string to store dataframe from single FSA
    temp_string = json.loads(response.text[2:])['DATA']
    
    # Convert temporary string into a temporary dataframe
    temp_df = pd.DataFrame.from_records(temp_string)
    
    # Assigning columns in temporary dataframe
    temp_df.columns = json.loads(response.text[2:])['COLUMNS']
    
    # Creating a single row with data of a single FSA
    single_income_row = temp_df['T_DATA_DONNEE']
    single_income_row = pd.DataFrame(single_income_row).T
    single_income_row.columns = single_data_columns
    
    # Populating the dataframe with the single row
    income_data = income_data.append(single_income_row, ignore_index = True)
    

# Add FSA value to to final table
income_data.insert(loc=0, column = "FSA", value = list_of_FSA)

print(f"Shape of Income Data: {income_data.shape}")


Shape of Income Data: (98, 212)


In [21]:
income_data = income_data[['FSA', '    Median employment income in 2015 for full-year full-time workers ($)']]
income_data.columns = ['FSA', 'Median Income']

## Final Income Data

In [22]:
income_data

,FSA,Median Income
0,M5V,69202.0
1,M5T,48942.0
2,M6J,57063.0
3,M6G,59052.0
4,M5S,66185.0
...,...,...
93,M2P,82561.0
94,M9R,49634.0
95,M2L,64489.0
96,L6A,60091.0


# Commute Data

## Data On One FSA Value

In [23]:
# Parameters for Mobility; One postal code only
parameters = {
"lang": "E",                     # Requested Data in English
"dguid": "2016A0011" + "V7C",    # Geographic Region
"topic": 8,                     # 12 = Mobility
"notes": 0,                      # Foot Notes
"stat": 1                        # Count (0) or Percentages (1)
}

# Get response
response = requests.get(url, params = parameters)

# Load data as text (disregard first two // symbols)
x = json.loads(response.text[2:])['DATA']

# Save text data as dataframe (for V7C)
df = pd.DataFrame.from_records(x)

# Add columns to the dataframe (for V7C)
df.columns = json.loads(response.text[2:])['COLUMNS']

# Rows that we want to turn into column names
single_data_columns = df['TEXT_NAME_NOM'].values

# Actual data values we want to fit under the column names.
single_data = df['T_DATA_DONNEE'] 

# Turn them into a row
single_data = pd.DataFrame(single_data).T

# Add column names
single_data.columns = single_data_columns

single_data

,Total - Commuting destination for the employed labour force aged 15 years and over in private households with a usual place of work - 25% sample data,Commute within census subdivision (CSD) of residence,Commute to a different census subdivision (CSD) within census division (CD) of residence,Commute to a different census subdivision (CSD) and census division (CD) within province or territory of residence,Commute to a different province or territory,Total - Main mode of commuting for the employed labour force aged 15 years and over in private households with a usual place of work or no fixed workplace address - 25% sample data,"Car, truck, van - as a driver","Car, truck, van - as a passenger",Public transit,Walked,...,30 to 44 minutes,45 to 59 minutes,60 minutes and over,Total - Time leaving for work for the employed labour force aged 15 years and over in private households with a usual place of work or no fixed workplace address - 25% sample data,Between 5 a.m. and 5:59 a.m.,Between 6 a.m. and 6:59 a.m.,Between 7 a.m. and 7:59 a.m.,Between 8 a.m. and 8:59 a.m.,Between 9 a.m. and 11:59 a.m.,Between 12 p.m. and 4:59 a.m.
T_DATA_DONNEE,100.0,57.0,41.9,0.6,0.6,100.0,68.9,6.3,18.7,3.5,...,25.3,11.4,8.8,100.0,3.9,11.6,21.4,24.7,22.7,15.6


## Data On All FSA Values

In [24]:
# Empty dataframe with the column names we want:
commute_data = pd.DataFrame(columns = single_data_columns)

# URL for Statistics Canada API
url = "https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json"

# For every FSA value:
for fsa_code in list_of_FSA:

    # Update parameters for API request
    parameters = {
    "lang": "E",                     # Requested Data in English
    "dguid": "2016A0011" + fsa_code, # Geographic Region
    "topic": 8,                     # Commute = 8
    "notes": 0,                      # Foot Notes
    "stat": 0                        # Count (0) or Percentages (1)
    }
    
    # API Request
    response = requests.get(url, params = parameters)
    
    # Create temporary string to store dataframe from single FSA
    temp_string = json.loads(response.text[2:])['DATA']
    
    # Convert temporary string into a temporary dataframe
    temp_df = pd.DataFrame.from_records(temp_string)
    
    # Assigning columns in temporary dataframe
    temp_df.columns = json.loads(response.text[2:])['COLUMNS']
    
    # Creating a single row with data of a single FSA
    single_commute_row = temp_df['T_DATA_DONNEE']
    single_commute_row = pd.DataFrame(single_commute_row).T
    single_commute_row.columns = single_data_columns
    
    # Populating the dataframe with the single row
    commute_data = commute_data.append(single_commute_row, ignore_index = True)
    

# Add FSA value to to final table
commute_data.insert(loc=0, column = "FSA", value = list_of_FSA)

print(f"Shape of Commute Data: {commute_data.shape}")


Shape of Commute Data: (98, 26)


In [25]:
commute_data.columns

Index(['FSA',
       'Total - Commuting destination for the employed labour force aged 15 years and over in private households with a usual place of work - 25% sample data',
       '  Commute within census subdivision (CSD) of residence',
       '  Commute to a different census subdivision (CSD) within census division (CD) of residence',
       '  Commute to a different census subdivision (CSD) and census division (CD) within province or territory of residence',
       '  Commute to a different province or territory',
       'Total - Main mode of commuting for the employed labour force aged 15 years and over in private households with a usual place of work or no fixed workplace address - 25% sample data',
       '  Car, truck, van - as a driver', '  Car, truck, van - as a passenger',
       '  Public transit', '  Walked', '  Bicycle', '  Other method',
       'Total - Commuting duration for the employed labour force aged 15 years and over in private households with a usual place of wor

In [26]:
# Only keep data on walking/driving:
commute_data['Commute:Drive'] = commute_data['  Car, truck, van - as a driver'] + \
                                commute_data['  Car, truck, van - as a passenger']
commute_data['Commute:Walk_Bus_Bike'] = commute_data['  Walked'] + \
                                        commute_data['  Bicycle'] + \
                                        commute_data['  Public transit']
commute_data = commute_data[['FSA', 'Commute:Drive', 'Commute:Walk_Bus_Bike']]

## Final Commute Data

In [27]:
commute_data.head()

,FSA,Commute:Drive,Commute:Walk_Bus_Bike
0,M5V,8540.0,25265.0
1,M5T,1255.0,6935.0
2,M6J,5445.0,12115.0
3,M6G,4870.0,11030.0
4,M5S,1365.0,5365.0


# Immigration Data

## Data On One FSA Value

In [28]:
# Parameters for Immigration; One postal code only
parameters = {
"lang": "E",                     # Requested Data in English
"dguid": "2016A0011" + "V7C",    # Geographic Region
"topic": 6,                     # 6 = Immigration & Citizenship
"notes": 0,                      # Foot Notes
"stat": 1                        # Count (0) or Percentages (1)
}

# Get response
response = requests.get(url, params = parameters)

# Load data as text (disregard first two // symbols)
x = json.loads(response.text[2:])['DATA']

# Save text data as dataframe (for V7C)
df = pd.DataFrame.from_records(x)

# Add columns to the dataframe (for V7C)
df.columns = json.loads(response.text[2:])['COLUMNS']

# Rows that we want to turn into column names
single_data_columns = df['TEXT_NAME_NOM'].values

# Actual data values we want to fit under the column names.
single_data = df['T_DATA_DONNEE'] 

# Turn them into a row
single_data = pd.DataFrame(single_data).T

# Add column names
single_data.columns = single_data_columns

single_data

,Total - Citizenship for the population in private households - 25% sample data,Canadian citizens,Canadian citizens aged under 18,Canadian citizens aged 18 and over,Not Canadian citizens,Total - Immigrant status and period of immigration for the population in private households - 25% sample data,Non-immigrants,Immigrants,Before 1981,1981 to 1990,...,First generation,Second generation,Third generation or more,Total - Admission category and applicant type for the immigrant population in private households who landed between 1980 and 2016 - 25% sample data,Economic immigrants,Principal applicants,Secondary applicants,Immigrants sponsored by family,Refugees,Other immigrants
T_DATA_DONNEE,100.0,80.6,15.1,65.5,19.4,100.0,34.2,63.0,8.3,6.9,...,66.4,21.4,12.2,100.0,74.1,24.3,49.8,19.9,3.8,2.2


## Data On All FSA Value

In [29]:
# Empty dataframe with the column names we want:
immigration_data = pd.DataFrame(columns = single_data_columns)

# URL for Statistics Canada API
url = "https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json"

# For every FSA value:
for fsa_code in list_of_FSA:

    # Update parameters for API request
    parameters = {
    "lang": "E",                     # Requested Data in English
    "dguid": "2016A0011" + fsa_code, # Geographic Region
    "topic": 6,                      # Immigration = 6
    "notes": 0,                      # Foot Notes
    "stat": 0                        # Count (0) or Percentages (1)
    }
    
    # API Request
    response = requests.get(url, params = parameters)
    
    # Create temporary string to store dataframe from single FSA
    temp_string = json.loads(response.text[2:])['DATA']
    
    # Convert temporary string into a temporary dataframe
    temp_df = pd.DataFrame.from_records(temp_string)
    
    # Assigning columns in temporary dataframe
    temp_df.columns = json.loads(response.text[2:])['COLUMNS']
    
    # Creating a single row with data of a single FSA
    single_immigration_row = temp_df['T_DATA_DONNEE']
    single_immigration_row = pd.DataFrame(single_immigration_row).T
    single_immigration_row.columns = single_data_columns
    
    # Populating the dataframe with the single row
    immigration_data = immigration_data.append(single_immigration_row, ignore_index = True)
    

# Add FSA value to to final table
immigration_data.insert(loc=0, column = "FSA", value = list_of_FSA)

print(f"Shape of Immigration Data: {immigration_data.shape}")


Shape of Immigration Data: (98, 155)


In [30]:
# Only keep immigrat/non-immigrant data
immigration_data = immigration_data[['FSA','  Non-immigrants', '  Immigrants']]
immigration_data.columns = ['FSA', 'NonImmigrants', 'Immigrants']

## Final Immigration Data

In [31]:
immigration_data.head()

,FSA,NonImmigrants,Immigrants
0,M5V,28920.0,16245.0
1,M5T,8500.0,7610.0
2,M6J,20335.0,10225.0
3,M6G,20505.0,9780.0
4,M5S,7220.0,5120.0


# Housing Data

## Data On One FSA Value

In [32]:
# Parameters for Ethnic Origin; One postal code only
parameters = {
"lang": "E",                     # Requested Data in English
"dguid": "2016A0011" + "M4Y",    # Geographic Region
"topic": 5,                      # 5 = Housing
"notes": 0,                      # Foot Notes
"stat": 1                       # Count (0) or Percentages (1)
}

# Get response
response = requests.get(url, params = parameters)

# Load data as text (disregard first two // symbols)
x = json.loads(response.text[2:])['DATA']

# Save text data as dataframe (for V7C)
df = pd.DataFrame.from_records(x)

# Add columns to the dataframe (for V7C)
df.columns = json.loads(response.text[2:])['COLUMNS']

# Rows that we want to turn into column names
single_data_columns = df['TEXT_NAME_NOM'].values

# Actual data values we want to fit under the column names.
single_data = df['T_DATA_DONNEE'] 

# Turn them into a row
single_data = pd.DataFrame(single_data).T

# Add column names
single_data.columns = single_data_columns

single_data.T[:30]

,T_DATA_DONNEE
Total - Private households by tenure - 25% sample data,100.0
Owner,27.2
Renter,72.8
Band housing,0.0
Total - Occupied private dwellings by condominium status - 25% sample data,100.0
Condominium,48.1
Not condominium,51.9
Total - Occupied private dwellings by number of bedrooms - 25% sample data,100.0
No bedrooms,6.3
1 bedroom,60.3


## Data On All FSA Values

In [33]:
# Empty dataframe with the column names we want:
housing_data = pd.DataFrame(columns = single_data_columns)

# URL for Statistics Canada API
url = "https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json"

# For every FSA value:
for fsa_code in list_of_FSA:

    # Update parameters for API request
    parameters = {
    "lang": "E",                     # Requested Data in English
    "dguid": "2016A0011" + fsa_code, # Geographic Region
    "topic": 5,                      # Housing = 5
    "notes": 0,                      # Foot Notes
    "stat": 0                        # Count (0) or Percentages (1)
    }
    
    # API Request
    response = requests.get(url, params = parameters)
    
    # Create temporary string to store dataframe from single FSA
    temp_string = json.loads(response.text[2:])['DATA']
    
    # Convert temporary string into a temporary dataframe
    temp_df = pd.DataFrame.from_records(temp_string)
    
    # Assigning columns in temporary dataframe
    temp_df.columns = json.loads(response.text[2:])['COLUMNS']
    
    # Creating a single row with data of a single FSA
    single_housing_row = temp_df['T_DATA_DONNEE']
    single_housing_row = pd.DataFrame(single_housing_row).T
    single_housing_row.columns = single_data_columns
    
    # Populating the dataframe with the single row
    housing_data = housing_data.append(single_housing_row, ignore_index = True)
    

# Add FSA value to to final table
housing_data.insert(loc=0, column = "FSA", value = list_of_FSA)

print(f"Shape of Housing Data: {housing_data.shape}")


Shape of Housing Data: (98, 67)


In [34]:
# TO DO: Keep: Owner/Renters, Median of rent & median of house buy, 
housing_data = housing_data[['FSA', '  Owner' , '  Renter', '  Median monthly shelter costs for rented dwellings ($)', '  Median monthly shelter costs for owned dwellings ($)']]
housing_data.columns = ['FSA', 'House_Owner_Pct', 'House_Renter_Pct', 'Median_Monthly_Rent', 'Median_Monthly_Buy']

## Final Housing Data

In [35]:
housing_data

,FSA,House_Owner_Pct,House_Renter_Pct,Median_Monthly_Rent,Median_Monthly_Buy
0,M5V,13265.0,17455.0,1715.0,1886.0
1,M5T,2510.0,6035.0,1101.0,1410.0
2,M6J,7355.0,7760.0,1402.0,1683.0
3,M6G,6640.0,6775.0,1279.0,1328.0
4,M5S,3025.0,4930.0,1686.0,1442.0
...,...,...,...,...,...
93,M2P,2225.0,800.0,1430.0,1710.0
94,M9R,6135.0,6240.0,1205.0,1056.0
95,M2L,2835.0,1160.0,1391.0,1928.0
96,L6A,22485.0,2345.0,1687.0,2077.0


# Combining All Demographic Data

With the following datasets:

- `visible_minority_data`
- `population_data`
- `income_data`
- `commute_data`
- `immigration_data`
- `housing_data`

I can combine all of them to create one large dataset. 

In [36]:
# Looking at the shape
print(f"Visible Minority Data Shape: {visible_minority_data.shape}")
print(f"Population Data Shape:       {population_data.shape}")
print(f"Income Data Shape:           {income_data.shape}")
print(f"Commute Data Shape:          {commute_data.shape}")
print(f"Immigration Data Shape:      {immigration_data.shape}")
print(f"Housing Data Shape:          {housing_data.shape}")

Visible Minority Data Shape: (98, 13)
Population Data Shape:       (98, 7)
Income Data Shape:           (98, 2)
Commute Data Shape:          (98, 3)
Immigration Data Shape:      (98, 3)
Housing Data Shape:          (98, 5)


In [37]:
# Combining all dataframes on 'FSA'
mask1 = pd.merge(visible_minority_data, population_data, on = 'FSA')
mask2 = pd.merge(mask1, income_data, on = 'FSA')
mask3 = pd.merge(mask2, commute_data, on = 'FSA')
mask4 = pd.merge(mask3, immigration_data, on = 'FSA')
demographic_data = pd.merge(mask4, housing_data, on = 'FSA')

In [38]:
demographic_data.head()

,FSA,Visible_Minority_Population_Pct,South_Asian_Pct,Chinese_Pct,Black_Pct,Filipino_Pct,Latin American_Pct,Arab_Pct,South_East_Asian_Pct,West_Asian_Pct,...,Age:60plus,Median Income,Commute:Drive,Commute:Walk_Bus_Bike,NonImmigrants,Immigrants,House_Owner_Pct,House_Renter_Pct,Median_Monthly_Rent,Median_Monthly_Buy
0,M5V,20745.0,4395.0,6290.0,2560.0,1020.0,1355.0,1000.0,605.0,1030.0,...,3045.0,69202.0,8540.0,25265.0,28920.0,16245.0,13265.0,17455.0,1715.0,1886.0
1,M5T,10455.0,765.0,6045.0,880.0,450.0,330.0,290.0,615.0,190.0,...,3535.0,48942.0,1255.0,6935.0,8500.0,7610.0,2510.0,6035.0,1101.0,1410.0
2,M6J,9055.0,900.0,4225.0,1100.0,440.0,615.0,170.0,335.0,150.0,...,5210.0,57063.0,5445.0,12115.0,20335.0,10225.0,7355.0,7760.0,1402.0,1683.0
3,M6G,7605.0,740.0,2415.0,1120.0,570.0,720.0,155.0,355.0,155.0,...,7190.0,59052.0,4870.0,11030.0,20505.0,9780.0,6640.0,6775.0,1279.0,1328.0
4,M5S,6450.0,1045.0,2980.0,355.0,210.0,225.0,265.0,145.0,205.0,...,3015.0,66185.0,1365.0,5365.0,7220.0,5120.0,3025.0,4930.0,1686.0,1442.0


In [39]:
# Number of missing values
demographic_data.isna().sum().sum()

0

# Export Data

In [41]:
# Export to CSV

demographic_data.to_csv("cleaned_demographic_data.csv")